# Introdução
Após a análise exploratória feita no outro arquivo, será feito o Machine Learning com os dados e conclusões obtidas.

# 1. Carregamento de Dataset
Utilizaremos os mesmos métodos de carregamento do arquivo de análise exploratória

## a) Utilizar a API do Kaggle

In [ ]:
# Parte para baixar o arquivo utilizando a API do Kaggle
import kagglehub

# Baixando o arquivo
caminho = kagglehub.dataset_download("khushikyad001/electric-vehicle-analytics-dataset")

print("Caminho para o dataset:", caminho)

## b) Utilizar arquivo .csv já baixado

In [ ]:
# Parte para utilizar o arquivo já baixado
import os

# Começando o caminho vazio
caminho = None

# Procurando o arquivo no diretório atual
arquivos = os.listdir(".")
for arquivo in arquivos:
    if arquivo == "electric_vehicle_analytics.csv":
        caminho = os.path.join(".",arquivo)
        break

if caminho:
    print("Caminho para o dataset:", caminho)
else:
    print("Dataset não encontrado!")

# 2. Pré-tratamento de dados
Nesta parte será retirada a coluna de ID de veículos, por não ser útil para a IA e também as variáveis categóricas serão tratadas para que possam ser utilizadas pela IA. Por último os dados serão separados entre teste e treino.